# AMAZON PC REVIEWS - NLP Topic Modeling
---

In [117]:
########################################
## Standard Python Modules/ Functions ##
########################################
import numpy as np
import pandas as pd
import re
import pprint
from collections import defaultdict
from collections import Counter
import pickle


import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

######################
## Plotting Modules ##
######################
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

%matplotlib inline

############################
## Classification Modules ##
############################
from imblearn.over_sampling import ADASYN
from smote_variants import DBSMOTE
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve,f1_score, fbeta_score, confusion_matrix, make_scorer

#########################################
## Natural Language Processing Modules ##
#########################################
import textstat
import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim import corpora, models, similarities, matutils
from gensim.models import Word2Vec

##########################################
## Clustering & Dimensionanlity Modules ##
##########################################
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import DBSCAN, KMeans, MeanShift

---

# Amazon PC Sub-Category (Laptops):
In order to improve results of topic modeling, focus on finding topics relevant to reviews that are for laptops only. The "PC Category" from Amazon Reviews data contains many PC related accessory products.

In [54]:
## Import Pickled dataframe
laptops_df = pd.read_pickle('data_files/laptops_df.pkl')

In [59]:
# Separate majority and minority classes
df_majority = laptops_df[laptops_df['target']==0]
df_minority = laptops_df[laptops_df['target']==1]

In [60]:
df_majority.shape, df_minority.shape

((80548, 23), (5344, 23))

In [64]:
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=10000, # to reduce class imbalance
                                 random_state=42)  # reproducible results
    
# Combine minority class with downsampled majority class
laptop_downsampled_df = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
laptop_downsampled_df['target'].value_counts()

0    10000
1     5344
Name: target, dtype: int64

In [65]:
## Reset the index values after down sampling
laptop_downsampled_df.reset_index(drop=True, inplace=True)

---

# Text Pre-Processing:
    1. Establish Stop Words
    2. Define WordNet Lemmitizer with POS Tag

### World Lemmitizer Using "Parts of Speech" (POS) Tag

In [29]:
def get_pos_tag(word):
    """ 
    Determine POS tag for a word, and return it's first character.
    To be used in Lemmatize() function.
    """
    # Dictionary for "Parts of Speech" Tags
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    tag = pos_tag([word])[0][1][0].upper()
    
    return tag_dict.get(tag, wordnet.NOUN)
    

In [30]:
## Create class object for Lemmatization Tokenizer
## Uses WordNetLemmatizer

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(word, pos=get_pos_tag(word)) for word in word_tokenize(doc)]

### Define All Stop Words

In [31]:
## Define "english" stopwords
stop_words = set(stopwords.words('english'))

In [234]:
## Add additional words to stopwords set
stop_words = stop_words.union({'pc', 'computer','laptop','macbook','notebook','ultrabook', 'mac', 'window', 'xps',
                               'hp','asus','lenovo','acer', 'dell','apple', 'air', 'pro', 'tablet'})

In [235]:
## New list of stop words that are Lemmatized (to match lemma-tokenized bag of words)
## Count Vectorizer (for word vectorization) used below will have words Lemmatized as well 
stop_lem = WordNetLemmatizer()
lem_stopwords = [stop_lem.lemmatize(word, pos=get_pos_tag(word)) for word in stop_words]

---

# LDA Model:

### Count Vectorizer (for LDA):

In [66]:
# cv1 = CountVectorizer(stop_words=lem_stopwords, ngram_range=(1,2), tokenizer=LemmaTokenizer())

# doc_words = cv1.fit_transform(laptop_downsampled_df['clean_review_body']).transpose()

In [67]:
# ## Pickle cv1 CountVectorizer
# pickle_out = open('laptops_cv1.pkl', 'wb') 

# pickle.dump(cv1, pickle_out)                      
# pickle_out.close() 

In [68]:
# ## Pickle doc_words Matrix
# pickle_out = open('laptops_doc_words.pkl', 'wb') 

# pickle.dump(doc_words, pickle_out)                      
# pickle_out.close() 

In [213]:
## Open pickled cv1 CountVectorizer
pickle_in = open('laptops_cv1.pkl', 'rb') 

cv1 = pickle.load(pickle_in)

In [214]:
## Open pickled doc_words Matrix
pickle_in = open('laptops_doc_words.pkl', 'rb') 

doc_words = pickle.load(pickle_in)

### Convert to Gensim:
Convert `doc_words` matrix to a `gensim`-friendly object (Corpus).

In [215]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_words)

In [216]:
## Creating mapping of matrix row id (document) to a word
## v = row id, k= vocab
id2word = dict((v, k) for k, v in cv1.vocabulary_.items())

### Create Model:

In [217]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus, num_topics=5, id2word=id2word, passes=40)

2019-11-20 03:08:46,331 : INFO : using symmetric alpha at 0.2
2019-11-20 03:08:46,332 : INFO : using symmetric eta at 0.2
2019-11-20 03:08:46,396 : INFO : using serial LDA version on this node
2019-11-20 03:08:46,716 : INFO : running online (multi-pass) LDA training, 5 topics, 40 passes over the supplied corpus of 15344 documents, updating model once every 2000 documents, evaluating perplexity every 15344 documents, iterating 50x with a convergence threshold of 0.001000
2019-11-20 03:08:46,749 : INFO : PROGRESS: pass 0, at document #2000/15344
2019-11-20 03:08:47,893 : INFO : merging changes from 2000 documents into a model of 15344 documents
2019-11-20 03:08:48,061 : INFO : topic #0 (0.200): 0.008*"laptop" + 0.005*"get" + 0.004*"use" + 0.004*"work" + 0.003*"great" + 0.003*"like" + 0.003*"would" + 0.003*"screen" + 0.003*"well" + 0.002*"one"
2019-11-20 03:08:48,065 : INFO : topic #1 (0.200): 0.005*"laptop" + 0.005*"use" + 0.004*"work" + 0.003*"window" + 0.003*"one" + 0.003*"battery" + 0

2019-11-20 03:08:55,765 : INFO : topic #0 (0.200): 0.004*"great" + 0.003*"laptop" + 0.003*"love" + 0.002*"use" + 0.002*"get" + 0.002*"work" + 0.001*"well" + 0.001*"screen" + 0.001*"would" + 0.001*"like"
2019-11-20 03:08:55,772 : INFO : topic #1 (0.200): 0.000*"de" + 0.000*"use" + 0.000*"laptop" + 0.000*"battery" + 0.000*"work" + 0.000*"window" + 0.000*"la" + 0.000*"price" + 0.000*"one" + 0.000*"great"
2019-11-20 03:08:55,778 : INFO : topic #2 (0.200): 0.013*"laptop" + 0.008*"use" + 0.006*"get" + 0.005*"screen" + 0.005*"window" + 0.005*"one" + 0.005*"like" + 0.004*"work" + 0.004*"well" + 0.004*"keyboard"
2019-11-20 03:08:55,784 : INFO : topic #3 (0.200): 0.002*"get" + 0.002*"use" + 0.002*"laptop" + 0.001*"drive" + 0.001*"one" + 0.001*"screen" + 0.001*"hard" + 0.001*"keyboard" + 0.001*"key" + 0.001*"work"
2019-11-20 03:08:55,790 : INFO : topic #4 (0.200): 0.001*"laptop" + 0.001*"good" + 0.001*"great" + 0.000*"product" + 0.000*"use" + 0.000*"excellent" + 0.000*"recommend" + 0.000*"one" + 

2019-11-20 03:09:03,554 : INFO : topic #2 (0.200): 0.012*"laptop" + 0.008*"use" + 0.006*"get" + 0.005*"work" + 0.005*"screen" + 0.005*"one" + 0.004*"like" + 0.004*"window" + 0.004*"well" + 0.004*"good"
2019-11-20 03:09:03,562 : INFO : topic #3 (0.200): 0.000*"excelente" + 0.000*"shes" + 0.000*"excelent" + 0.000*"get" + 0.000*"laptop" + 0.000*"use" + 0.000*"drive" + 0.000*"one" + 0.000*"screen" + 0.000*"hard"
2019-11-20 03:09:03,568 : INFO : topic #4 (0.200): 0.000*"selleri recommend" + 0.000*"recommend selleri" + 0.000*"selleri" + 0.000*"recommend" + 0.000*"product love" + 0.000*"husband love" + 0.000*"product excellent" + 0.000*"problem wont" + 0.000*"ever deal" + 0.000*"product meet"
2019-11-20 03:09:03,572 : INFO : topic diff=0.056104, rho=0.321545
2019-11-20 03:09:03,674 : INFO : PROGRESS: pass 1, at document #12000/15344
2019-11-20 03:09:04,363 : INFO : merging changes from 2000 documents into a model of 15344 documents
2019-11-20 03:09:04,543 : INFO : topic #0 (0.200): 0.001*"lov

2019-11-20 03:09:11,534 : INFO : topic #2 (0.200): 0.012*"laptop" + 0.007*"use" + 0.006*"get" + 0.005*"screen" + 0.005*"one" + 0.004*"work" + 0.004*"like" + 0.004*"window" + 0.004*"well" + 0.004*"good"
2019-11-20 03:09:11,540 : INFO : topic #3 (0.200): 0.000*"excelente" + 0.000*"excelent" + 0.000*"slow expect" + 0.000*"excelente producto" + 0.000*"producto" + 0.000*"gracias" + 0.000*"buen producto" + 0.000*"recomendado" + 0.000*"shes" + 0.000*"muchas"
2019-11-20 03:09:11,544 : INFO : topic #4 (0.200): 0.000*"selleri" + 0.000*"selleri recommend" + 0.000*"recommend selleri" + 0.000*"realy" + 0.000*"husband love" + 0.000*"broke time" + 0.000*"item good" + 0.000*"back please" + 0.000*"excellent good" + 0.000*"pocket hold"
2019-11-20 03:09:11,548 : INFO : topic diff=0.039735, rho=0.306110
2019-11-20 03:09:11,596 : INFO : PROGRESS: pass 2, at document #8000/15344
2019-11-20 03:09:11,997 : INFO : merging changes from 2000 documents into a model of 15344 documents
2019-11-20 03:09:12,177 : INF

2019-11-20 03:09:19,232 : INFO : merging changes from 2000 documents into a model of 15344 documents
2019-11-20 03:09:19,413 : INFO : topic #0 (0.200): 0.000*"love love" + 0.000*"love" + 0.000*"exelente" + 0.000*"great lap" + 0.000*"son christmas" + 0.000*"gift daughter" + 0.000*"fast thin" + 0.000*"christmas love" + 0.000*"bought friend" + 0.000*"love lap"
2019-11-20 03:09:19,422 : INFO : topic #1 (0.200): 0.000*"de" + 0.000*"la" + 0.000*"el" + 0.000*"que" + 0.000*"en" + 0.000*"producto" + 0.000*"un" + 0.000*"muy" + 0.000*"e" + 0.000*"lo"
2019-11-20 03:09:19,431 : INFO : topic #2 (0.200): 0.011*"laptop" + 0.007*"use" + 0.006*"get" + 0.005*"screen" + 0.004*"one" + 0.004*"like" + 0.004*"window" + 0.004*"work" + 0.004*"well" + 0.003*"would"
2019-11-20 03:09:19,440 : INFO : topic #3 (0.200): 0.000*"excelente" + 0.000*"excelent" + 0.000*"excelente producto" + 0.000*"producto" + 0.000*"recomendado" + 0.000*"slow expect" + 0.000*"gracias" + 0.000*"excelent laptop" + 0.000*"muchas gracias" + 

...

2019-11-20 03:14:25,260 : INFO : topic diff=0.017166, rho=0.149617
2019-11-20 03:14:25,309 : INFO : PROGRESS: pass 36, at document #10000/15344
2019-11-20 03:14:25,706 : INFO : merging changes from 2000 documents into a model of 15344 documents
2019-11-20 03:14:25,884 : INFO : topic #0 (0.200): 0.000*"gift daughter" + 0.000*"get son" + 0.000*"exelente" + 0.000*"gift love" + 0.000*"school love" + 0.000*"apple airport" + 0.000*"lapto" + 0.000*"granddaughter love" + 0.000*"daughter school" + 0.000*"love excellent"
2019-11-20 03:14:25,887 : INFO : topic #1 (0.200): 0.001*"de" + 0.001*"la" + 0.000*"que" + 0.000*"el" + 0.000*"e" + 0.000*"en" + 0.000*"muy" + 0.000*"lo" + 0.000*"excelente" + 0.000*"un"
2019-11-20 03:14:25,894 : INFO : topic #2 (0.200): 0.011*"laptop" + 0.007*"use" + 0.006*"get" + 0.004*"screen" + 0.004*"one" + 0.004*"work" + 0.004*"like" + 0.004*"window" + 0.004*"well" + 0.003*"good"
2019-11-20 03:14:25,897 : INFO : topic #3 (0.200): 0.000*"excelente" + 0.000*"excelent" + 0.00

2019-11-20 03:14:33,106 : INFO : topic #3 (0.200): 0.000*"excelente" + 0.000*"excelent" + 0.000*"excelent laptop" + 0.000*"excelent product" + 0.000*"last drop" + 0.000*"despite good" + 0.000*"since x" + 0.000*"less moth" + 0.000*"restore whole" + 0.000*"x sl"
2019-11-20 03:14:33,111 : INFO : topic #4 (0.200): 0.000*"selleri recommend" + 0.000*"selleri" + 0.000*"recommend selleri" + 0.000*"broke time" + 0.000*"repair give" + 0.000*"program ever" + 0.000*"mine broke" + 0.000*"back please" + 0.000*"husband love" + 0.000*"ever deal"
2019-11-20 03:14:33,116 : INFO : topic diff=0.016728, rho=0.147970
2019-11-20 03:14:33,164 : INFO : PROGRESS: pass 37, at document #6000/15344
2019-11-20 03:14:33,563 : INFO : merging changes from 2000 documents into a model of 15344 documents
2019-11-20 03:14:33,740 : INFO : topic #0 (0.200): 0.000*"gift daughter" + 0.000*"get son" + 0.000*"exelente" + 0.000*"school love" + 0.000*"gift love" + 0.000*"daughter school" + 0.000*"lapto" + 0.000*"apple airport" + 

2019-11-20 03:14:40,912 : INFO : topic #1 (0.200): 0.000*"de" + 0.000*"la" + 0.000*"que" + 0.000*"el" + 0.000*"e" + 0.000*"en" + 0.000*"muy" + 0.000*"lo" + 0.000*"un" + 0.000*"excelente"
2019-11-20 03:14:40,918 : INFO : topic #2 (0.200): 0.011*"laptop" + 0.007*"use" + 0.006*"get" + 0.005*"screen" + 0.004*"one" + 0.004*"like" + 0.004*"window" + 0.004*"work" + 0.004*"well" + 0.003*"would"
2019-11-20 03:14:40,923 : INFO : topic #3 (0.200): 0.000*"excelente" + 0.000*"excelent" + 0.000*"excelent product" + 0.000*"excelent laptop" + 0.000*"excelente product" + 0.000*"wish cd" + 0.000*"since x" + 0.000*"last drop" + 0.000*"despite good" + 0.000*"less moth"
2019-11-20 03:14:40,927 : INFO : topic #4 (0.200): 0.000*"selleri recommend" + 0.000*"selleri" + 0.000*"recommend selleri" + 0.000*"broke time" + 0.000*"back please" + 0.000*"ever deal" + 0.000*"unit asus" + 0.000*"repair give" + 0.000*"program ever" + 0.000*"mine broke"
2019-11-20 03:14:40,931 : INFO : topic diff=0.063907, rho=0.147970
201

2019-11-20 03:14:44,319 : INFO : PROGRESS: pass 38, at document #12000/15344
2019-11-20 03:14:44,990 : INFO : merging changes from 2000 documents into a model of 15344 documents
2019-11-20 03:14:45,168 : INFO : topic #0 (0.200): 0.000*"gift daughter" + 0.000*"get son" + 0.000*"exelente" + 0.000*"gift love" + 0.000*"school love" + 0.000*"apple airport" + 0.000*"lapto" + 0.000*"granddaughter love" + 0.000*"daughter school" + 0.000*"love excellent"
2019-11-20 03:14:45,172 : INFO : topic #1 (0.200): 0.001*"de" + 0.000*"la" + 0.000*"que" + 0.000*"el" + 0.000*"e" + 0.000*"en" + 0.000*"muy" + 0.000*"lo" + 0.000*"excelente" + 0.000*"un"
2019-11-20 03:14:45,181 : INFO : topic #2 (0.200): 0.011*"laptop" + 0.007*"use" + 0.006*"get" + 0.005*"screen" + 0.004*"one" + 0.004*"window" + 0.004*"like" + 0.004*"work" + 0.004*"well" + 0.003*"good"
2019-11-20 03:14:45,184 : INFO : topic #3 (0.200): 0.000*"excelente" + 0.000*"excelent" + 0.000*"excelent product" + 0.000*"excelent laptop" + 0.000*"excelente p

2019-11-20 03:14:51,977 : INFO : topic #3 (0.200): 0.000*"excelente" + 0.000*"excelent" + 0.000*"excelent product" + 0.000*"excelent laptop" + 0.000*"last drop" + 0.000*"despite good" + 0.000*"since x" + 0.000*"less moth" + 0.000*"big ibm" + 0.000*"card mess"
2019-11-20 03:14:51,980 : INFO : topic #4 (0.200): 0.000*"selleri recommend" + 0.000*"selleri" + 0.000*"recommend selleri" + 0.000*"broke time" + 0.000*"husband love" + 0.000*"realy" + 0.000*"back please" + 0.000*"repair give" + 0.000*"program ever" + 0.000*"mine broke"
2019-11-20 03:14:51,984 : INFO : topic diff=0.016360, rho=0.144833
2019-11-20 03:14:52,034 : INFO : PROGRESS: pass 39, at document #8000/15344
2019-11-20 03:14:52,419 : INFO : merging changes from 2000 documents into a model of 15344 documents
2019-11-20 03:14:52,580 : INFO : topic #0 (0.200): 0.000*"gift daughter" + 0.000*"get son" + 0.000*"exelente" + 0.000*"gift love" + 0.000*"school love" + 0.000*"lapto" + 0.000*"daughter school" + 0.000*"granddaughter love" + 

In [218]:
# ## Pickle LDA Model
# pickle_out = open('laptop_lda_model.pkl', 'wb') 

# pickle.dump(lda, pickle_out)                      
# pickle_out.close() 

In [220]:
## Open pickled LDA Model
pickle_in = open('laptop_lda_model.pkl', 'rb') 

lda = pickle.load(pickle_in)

### Topic Space:

In [221]:
lda.print_topics()

2019-11-20 03:36:38,148 : INFO : topic #0 (0.200): 0.000*"gift daughter" + 0.000*"get son" + 0.000*"exelente" + 0.000*"gift love" + 0.000*"granddaughter love" + 0.000*"school love" + 0.000*"apple airport" + 0.000*"lapto" + 0.000*"daughter school" + 0.000*"love excellent"
2019-11-20 03:36:38,154 : INFO : topic #1 (0.200): 0.000*"de" + 0.000*"la" + 0.000*"que" + 0.000*"el" + 0.000*"e" + 0.000*"en" + 0.000*"muy" + 0.000*"lo" + 0.000*"un" + 0.000*"excelente"
2019-11-20 03:36:38,164 : INFO : topic #2 (0.200): 0.011*"laptop" + 0.007*"use" + 0.006*"get" + 0.005*"screen" + 0.004*"one" + 0.004*"like" + 0.004*"window" + 0.004*"work" + 0.004*"well" + 0.003*"would"
2019-11-20 03:36:38,170 : INFO : topic #3 (0.200): 0.000*"excelente" + 0.000*"excelent" + 0.000*"excelent product" + 0.000*"excelent laptop" + 0.000*"excelente product" + 0.000*"wish cd" + 0.000*"since x" + 0.000*"last drop" + 0.000*"despite good" + 0.000*"less moth"
2019-11-20 03:36:38,175 : INFO : topic #4 (0.200): 0.000*"recommend se

[(0,
  '0.000*"gift daughter" + 0.000*"get son" + 0.000*"exelente" + 0.000*"gift love" + 0.000*"granddaughter love" + 0.000*"school love" + 0.000*"apple airport" + 0.000*"lapto" + 0.000*"daughter school" + 0.000*"love excellent"'),
 (1,
  '0.000*"de" + 0.000*"la" + 0.000*"que" + 0.000*"el" + 0.000*"e" + 0.000*"en" + 0.000*"muy" + 0.000*"lo" + 0.000*"un" + 0.000*"excelente"'),
 (2,
  '0.011*"laptop" + 0.007*"use" + 0.006*"get" + 0.005*"screen" + 0.004*"one" + 0.004*"like" + 0.004*"window" + 0.004*"work" + 0.004*"well" + 0.003*"would"'),
 (3,
  '0.000*"excelente" + 0.000*"excelent" + 0.000*"excelent product" + 0.000*"excelent laptop" + 0.000*"excelente product" + 0.000*"wish cd" + 0.000*"since x" + 0.000*"last drop" + 0.000*"despite good" + 0.000*"less moth"'),
 (4,
  '0.000*"recommend selleri" + 0.000*"selleri recommend" + 0.000*"selleri" + 0.000*"broke time" + 0.000*"back please" + 0.000*"ever deal" + 0.000*"unit asus" + 0.000*"repair give" + 0.000*"program ever" + 0.000*"mine broke"')

---

# Latent Semantic Analysis (LSA) w/ Count Vectorizer:

In [236]:
cv2 = CountVectorizer(stop_words=lem_stopwords, tokenizer=LemmaTokenizer())
cv2

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=['in', 'whom', 'ain', 'other', 'where', 'very', 'be',
                            'yourself', 'who', 'our', 'until', 'should', 'he',
                            'or', 'some', 'mac', 'macbook', 'mustn', 'all',
                            'have', 'be', 'haven', 'laptop', 'own', "she's",
                            'herself', 'most', 'so', 'ours', "it's", ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<__main__.LemmaTokenizer object at 0x7f1352bc3cf8>,
                vocabulary=None)

In [237]:
## Focus on Laptop Reviews that were deemed "Helpful"
helpful_laptop_reviews = laptop_downsampled_df.loc[(laptop_downsampled_df['target']==1) 
                                                  & (laptop_downsampled_df['helpfullness_ratio'] >0.9)
                                                  & (laptop_downsampled_df['total_votes'] >50)]['clean_review_body']

In [238]:
## Tranform laptop reviews into word vectors
doc_words2 = cv2.fit_transform(helpful_laptop_reviews)
doc_words2

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.



<621x11367 sparse matrix of type '<class 'numpy.int64'>'
	with 129898 stored elements in Compressed Sparse Row format>

In [239]:
## Apply LSA to word vectors (Singular Value Decomposition)
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(doc_words2)
lsa.explained_variance_ratio_

array([0.21115552, 0.0439523 , 0.04040871, 0.02126175, 0.01899862])

In [240]:
## Function to extract topics from LSA
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [241]:
display_topics(lsa, cv2.get_feature_names(), 8)


Topic  0
use, get, screen, like, drive, one, gb, battery

Topic  1
x, mhz, cpu, ddr, battery, run, control, dual

Topic  2
gb, drive, model, ssd, inch, x, large, memory

Topic  3
use, port, thing, screen, work, usb, drive, hard

Topic  4
drive, usb, game, update, touch, driver, system, graphic


In [242]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = helpful_laptop_reviews,
             columns = ["c1","c2","c3","c4","c5"])
Vt

c1       c2  \
clean_review_body                                                       
i got this after a real odyssey searching for a...  15.56473 -0.35522   
on paper this laptop has a lot going for it a r...  12.31061 -0.37334   
just bought a unit from a local store i agree w...  17.62830  0.58044   
acer isnt an unfamiliar brand around our house ...  15.68340  1.84214   
the asus avd-ab is simply overall an excellent ...  14.94281  0.18919   
...                                                      ...      ...   
asus began the netbook era with a very small co...  17.92992  1.68442   
st solid construction little to no keyboard fle...  18.16815  4.50644   
amazing notebook for under fast shipping from a...  13.39723  0.38802   
i was looking for a lightweight lb or less touc...  22.85064 -1.22980   
well its expensive more expensive than any tabl...  21.50725  3.02659   

                                                         c3       c4       c5  
clean_review_body                                                              
i got this after a real odyssey searching for a... -3.67132  2.17011 -2.20560  
on paper this laptop has a lot going for it a r...  0.58309  2.95031  3.82166  
just bought a unit from a local store i agree w... -6.01376  1.57207  2.21620  
acer isnt an unfamiliar brand around our house ... -5.57944 -4.52295 -0.31121  
the asus avd-ab is simply overall an excellent ...  4.32073 -3.28880  1.47354  
...                                                     ...      ...      ...  
asus began the netbook era with a very small co... -0.63019 -1.22691 -1.56949  
st solid construction little to no keyboard fle... -4.97872 -1.80347  3.57251  
amazing notebook for under fast shipping from a... -1.28099  0.36561  0.09482  
i was looking for a lightweight lb or less touc... -7.10761  0.96281  0.39127  
well its expensive more expensive than any tabl... -0.34999  0.04422  2.61705  

[621 rows x 5 columns]

---

# Latent Semantic Analysis (LSA) w/ TF-IDF:

In [243]:
tfidf = TfidfVectorizer(stop_words=lem_stopwords, ngram_range=(1,1), tokenizer=LemmaTokenizer())
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['in', 'whom', 'ain', 'other', 'where', 'very', 'be',
                            'yourself', 'who', 'our', 'until', 'should', 'he',
                            'or', 'some', 'mac', 'macbook', 'mustn', 'all',
                            'have', 'be', 'haven', 'laptop', 'own', "she's",
                            'herself', 'most', 'so', 'ours', "it's", ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<__main__.LemmaTokenizer object at 0x7f13563f4f60>,
                use_idf=True, vocabulary=None)

In [244]:
## Tranform laptop reviews into word vectors
doc_words3 = tfidf.fit_transform(helpful_laptop_reviews)
doc_words3

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.



<621x11367 sparse matrix of type '<class 'numpy.float64'>'
	with 129898 stored elements in Compressed Sparse Row format>

In [245]:
## Apply LSA to word vectors (Singular Value Decomposition)
lsa2 = TruncatedSVD(5)
doc_topic = lsa2.fit_transform(doc_words3)
lsa2.explained_variance_ratio_

array([0.01053776, 0.01147602, 0.00829814, 0.00768231, 0.00749289])

In [246]:
display_topics(lsa2, tfidf.get_feature_names(), 10)


Topic  0
use, get, screen, like, drive, gb, one, battery, keyboard, well

Topic  1
gb, drive, model, ssd, memory, core, hard, usb, ram, port

Topic  2
game, graphic, play, core, battery, look, fps, cpu, price, great

Topic  3
new, amazon, model, purchase, go, issue, buy, update, machine, driver

Topic  4
game, driver, update, graphic, setting, fps, play, install, instal, system


In [247]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = helpful_laptop_reviews,
             columns = ["c1","c2","c3","c4","c5"])
Vt

c1       c2       c3  \
clean_review_body                                                               
i got this after a real odyssey searching for a...  0.30392 -0.02688  0.04068   
on paper this laptop has a lot going for it a r...  0.32110  0.06449 -0.03695   
just bought a unit from a local store i agree w...  0.32821 -0.12810 -0.09202   
acer isnt an unfamiliar brand around our house ...  0.34430 -0.05443  0.16449   
the asus avd-ab is simply overall an excellent ...  0.33284  0.14347 -0.00808   
...                                                     ...      ...      ...   
asus began the netbook era with a very small co...  0.31181  0.02196 -0.04501   
st solid construction little to no keyboard fle...  0.29990 -0.06714  0.05830   
amazing notebook for under fast shipping from a...  0.25647 -0.04300 -0.09490   
i was looking for a lightweight lb or less touc...  0.43795 -0.13570 -0.03846   
well its expensive more expensive than any tabl...  0.31863  0.00779 -0.00716   

                                                         c4       c5  
clean_review_body                                                     
i got this after a real odyssey searching for a... -0.14784 -0.19202  
on paper this laptop has a lot going for it a r... -0.20172  0.02293  
just bought a unit from a local store i agree w... -0.17448  0.00732  
acer isnt an unfamiliar brand around our house ...  0.01643 -0.01469  
the asus avd-ab is simply overall an excellent ...  0.07704  0.10352  
...                                                     ...      ...  
asus began the netbook era with a very small co... -0.04897  0.00491  
st solid construction little to no keyboard fle... -0.05277  0.16872  
amazing notebook for under fast shipping from a...  0.01951  0.09479  
i was looking for a lightweight lb or less touc... -0.03881 -0.02063  
well its expensive more expensive than any tabl... -0.11013  0.02638  

[621 rows x 5 columns]